In [1]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 0.0/2.3 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.3 GB 178.6 kB/s eta 3:38:42
     ---------------------------------------- 0.0/2.3 GB 178.6 kB/s eta 3:38:42
     ---------------------------------------- 0.0/2.3 GB 178.6 kB/s eta 3:38:42
     ---------------------------------------- 0.0/2.3 GB 178.6 kB/s eta 3:38:42
     ---------------------------------------- 0.0/2.3 GB 387.0 kB/s eta 1:40:56
     ---------------------------------------- 0.0/2.3 GB 387.0 kB/s eta 1:40:56
     ---------------------------------------- 0.0/2.3 GB 387.0 kB/s eta 1:40:56
     ---------------------------------------- 0.0/2.3 GB 387.0 kB/s eta 1:40:56
     ------------

In [2]:
import torch
DEVICE = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')
print("Using PyTorch version: {}, Device: {}".format(torch.__version__, DEVICE))

Using PyTorch version: 2.0.0+cu117, Device: cuda


In [3]:
!pip install opencv-python
!pip install tqdm
!pip install matplotlib

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.4/38.8 MB 23.9 MB/s eta 0:00:02
   - -------------------------------------- 1.0/38.8 MB 16.5 MB/s eta 0:00:03
   - -------------------------------------- 1.6/38.8 MB 14.7 MB/s eta 0:00:03
   -- ------------------------------------- 2.2/38.8 MB 13.7 MB/s eta 0:00:03
   -- ------------------------------------- 2.7/38.8 MB 13.3 MB/s eta 0:00:03
   --- ------------------------------------ 3.3/38.8 MB 13.0 MB/s eta 0:00:03
   --- ------------------------------------ 3.4/38.8 MB 11.4 MB/s eta 0:00:04
   --- ------------------------------------ 3.6/38.8 MB 10.4 MB/s eta 0:00:04
   --- ------------------------------------ 3.8/38.8 MB 9.8 MB/s eta 0:00:04
   ---- ----------------------------------- 4.4/38.8 MB 10.1 MB/s eta 0:00:04
   ----- ---------------------------------- 5.0/38.8 MB 10.3 MB/s eta 0:00:04
   ----- ---------------------------------- 5.6/38.8 MB 10.5 MB/s eta 0:0

In [4]:
#@title Utils (수정 필요 x)

import cv2
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

def train(model, train_loader, optimizer, scheduler):

    model.train()
    tqdm_bar = tqdm(enumerate(train_loader))
    for batch_idx, (image, label) in tqdm_bar:
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        tqdm_bar.set_description("Epoch {} - train loss: {:.6f}".format(epoch, loss.item()))
    scheduler.step()


def evaluate(model, test_loader):

    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in tqdm(test_loader):
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

def gaussian_smoothing(image, filter_size=3, sigma=1.0):

    center = (filter_size-1)/2
    gaussian_filter = np.zeros((filter_size, filter_size))
    for row in range(filter_size):
        for col in range(filter_size):
            gaussian_filter[row, col] = np.exp((-(row-center) ** 2 - (col-center) ** 2) / (2 * sigma ** 2)) / (2 * np.pi * sigma ** 2)
    gaussian_filter = gaussian_filter / np.sum(gaussian_filter)
    image = cv2.filter2D(image, -1, gaussian_filter)
    return image

def color_jitter(image):

    image = cv2.convertScaleAbs(image, alpha=1.5, beta=20)
    return image

class CustomDataset(Dataset):
    def __init__(self, train, prob=0.5, data_dir="./CIFAR_10"):

        self.data = datasets.CIFAR10(root=data_dir, train=train, download=True)
        self.prob = prob

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):

        return self.data[idx]

    def transform(self, image):

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if np.random.uniform() < self.prob:
          a = np.random.choice(2,1)
          if a == 0:
            image = gaussian_smoothing(image)
          else:
            image = color_jitter(image)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def collate_fn(self, data):

        batch_x, batch_y = [], []

        for x, y in data:
          x = np.array(x)
          x = self.transform(x)
          x = x / 255.0
          x = np.array([x[:,:,i] for i in range(3)])
          batch_x.append(x)
          batch_y.append(y)
        batch_x = torch.FloatTensor(batch_x)
        batch_y = torch.LongTensor(batch_y)

        return batch_x, batch_y

In [5]:
#@title Dataloader (수정 필요 x)

train_dataset = CustomDataset(train=True, prob=0.5)
test_dataset = CustomDataset(train=False, prob=0.5)

BATCH_SIZE = 64

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          collate_fn=train_dataset.collate_fn)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         collate_fn=test_dataset.collate_fn)

100.0%


Extracting ./CIFAR_10\cifar-10-python.tar.gz to ./CIFAR_10
Files already downloaded and verified


In [17]:
class ConvNet(nn.Module):

    def __init__(self):

        super(ConvNet, self).__init__()

        ### COMPLETE HERE ###

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=512, kernel_size=3, padding=1, padding_mode='zeros')
        self.b1 = nn.BatchNorm2d(512)
        self.p = 0.1
        self.dropout = nn.Dropout2d(p=self.p)
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1, padding_mode='zeros')
        self.b2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1, padding_mode='zeros')
        self.b3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1, padding_mode='zeros')
        self.b4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1, padding_mode='zeros')
        self.b5 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(8 * 8 * 32, 32)
        self.bfc1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 32)
        self.bfc2 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 10)

        ### COMPLETE HERE ###

    def forward(self, x):
        x = self.conv1(x)
        x = self.b1(x)
        x = F.gelu(x)

        x = self.conv2(x)
        x = self.b2(x)
        x = self.dropout(x)
        x = F.gelu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.b3(x)
        x = self.dropout(x)
        x = F.gelu(x)

        x = self.conv4(x)
        x = self.b4(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv5(x)
        x = self.b5(x)
        x = F.relu(x)

        # FC
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.bfc1(x)

        x = self.fc2(x)
        x = F.relu(x)
        x = self.bfc2(x)

        x = self.fc3(x)

        return x

In [19]:
model = ConvNet().to(DEVICE)

### COMPLETE HERE ###
# 다양한 hyperparameter를 시도해볼 것
# optimizer를 바꿔볼 것
# learning rate scheduler를 추가해볼 것

EPOCHS = 30
lr = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 150)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 50, 3)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, scheduler)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tModel: ConvNet, \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

### COMPLETE HERE ###

0it [00:00, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]


[EPOCH: 1], 	Model: ConvNet, 	Test Loss: 0.0175, 	Test Accuracy: 60.71 % 



0it [00:00, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]


[EPOCH: 29], 	Model: ConvNet, 	Test Loss: 0.0102, 	Test Accuracy: 82.21 % 



0it [00:00, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]


[EPOCH: 30], 	Model: ConvNet, 	Test Loss: 0.0106, 	Test Accuracy: 81.61 % 

